In [3]:
from docplex.mp.model import Model
import pandas as pd

# sava datas in a matrix of matrixes
first_data = [[0, 67, 23, 89], [56, 0, 45, 23], [12, 34, 0, 78], [78, 90, 12, 0]]

second_data = [[10, 5, 5, 5], [5, 5, 10, 5], [5, 5, 5, 10], [6, 10, 8, 10]]

third_data = [[0, 78, 12, 34], [23, 0, 67, 45], [89, 23, 0, 56], [45, 78, 90, 0]]

data = [first_data, second_data, third_data]

# parameters
c = 4  # number of nodes
d = [8, 9, 10, 11]  # time intervals
tw = [[8 , 12], [8, 12], [8, 12]]  # time windows
v = 2  # number of vehicles
q = [0, 10, 10, -10]  # quantity of goods to collect/deliver
Q = 100  # maximum capacity of the vehicle
t_mv = 0.5  # time to move one item in/out from a vehicle [min]
M = 100000  # big M
tot_q = 0  # total quantity of goods to collect/delivery
del_q = 0  # quantity of goods to deliver

for good in q:
    tot_q += abs(good)

tot_t_mv = tot_q * t_mv

print("Total time required to collect/deliver all goods:", tot_t_mv, "minutes")

for good in q:
    if good < 0:
        del_q += good
del_q = abs(del_q)

# define model
mdl = Model("CVRP")

# decision variables

# Crete binary variable  4D
x = mdl.binary_var_dict(
    ((i, j, t, k) for i in range(c) for j in range(c) for t in range(len(d) - 1) for k in range(v)),
    name="x"
)
T = mdl.continuous_var_matrix(c + 1, v,name="t")
C = mdl.continuous_var_matrix(c,v, name="C")

# nodes constraints
for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[0, j, t, k] for j in range(1, c)) for t in range(len(d) - 1)) == 1   #TO CHANGE FROM ==(it was a test to make boot vehicle works) to <=
    )


for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[j, 0, t, k] for j in range(1, c)) for t in range(len(d) - 1)) <= 1
    )


for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j) for t in range(len(d) - 1)
            ) for k in range(v))
        == 1
    )

for k in range(v):
    for j in range(1, c):
        mdl.add_constraint(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j) for t in range(len(d) - 1)
            )
            == mdl.sum(
                mdl.sum(x[j, i, t, k] for i in range(c) if i != j) for t in range(len(d) - 1)
            )
        )

for k in range(v):
    for t in range(len(d) - 1):
        mdl.add_constraint(T[0, k] >= d[t] * 60 * mdl.sum(x[0, j, t, k] for j in range(1, c)))

for k in range(v):
    for i in range(c):
        for j in range(1, c):
            for t in range(len(d) - 1):
                if i != j:
                    mdl.add_constraint((1 - x[i, j, t, k]) * M + T[j, k] - T[i, k] >= data[t][i][j])

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint((1 - x[i, 0, t, k]) * M + T[c, k] - T[i, k] >= data[t][i][j])

for k in range(v):
    for i in range(1, c):
        mdl.add_constraint(tw[i - 1][0] * 60 <= T[i, k])

for k in range(v):
    for i in range(1, c):
        mdl.add_constraint(T[i, k] <= tw[i - 1][1] * 60)

# quantity constraints
for k in range(v):
    for t in range(len(d) - 1):
        for i in range(c):
            for j in range(1, c):
                if i != j:
                    mdl.add_constraint(C[i, k] - C[j, k] + q[j] <= (1 - x[i, j, t, k]) * M)
                    mdl.add_constraint(C[j, k] - C[i, k] - q[j] <= (1 - x[i, j, t, k]) * M)

for k in range(v):
    for i in range(c):
        mdl.add_constraint(max(0, q[i]) <= C[i, k])
        mdl.add_constraint(C[i, k] <= min(Q, Q + q[i]))

for k in range(v):
    del_qi=0
    for j in range(1,c):
        if q[j] < 0:
            del_qi += q[j] * mdl.sum(mdl.sum(x[i, j, t, k] for i in range(c))for t in range(len(d)-1))
    mdl.add_constraint(C[0, k] >= -del_qi )

# objective function
mdl.minimize(
    mdl.sum(
        mdl.sum(
            mdl.sum(mdl.sum(x[i, j, t, k] * data[t][i][j] for i in range(c)) for j in range(c))
            for t in range(len(d) - 1)
        ) for k in range(v)
    ) 
)

sol = mdl.solve()

if sol is None:
    print("No solution found")

print(sol.objective_value + tot_t_mv)


for k in range(v):
    print(f"\nvehicle {k}:")
    for t in range(len(d) - 1):
        print(f"\nRoute for time interval {t}:")
        for start in range(c):
            if start == 0:
                for end in range(c):
                    if sol.get_value(x[start, end, t, k]) == 1:
                        route = [(start, sol.get_value(T[start, k]))]
                        next_node = end
                        while next_node != 0:
                            route.append((next_node, sol.get_value(T[next_node, k])))
                            for next_end in range(c):
                                if sol.get_value(x[next_node, next_end, t, k]) == 1:
                                    next_node = next_end
                                    break
                        route.append(
                            (0, sol.get_value(T[c, k]))
                        )  # Use T[c] for the return to depot
                        if route[0][0] == 0 and route[-1][0] == 0:
                            print(
                                " -> ".join(
                                    f"{node} (arrivo: {arrival:.2f})"
                                    for node, arrival in route
                                )
                            )
for k in range(v):
    print(f"\nvehicle {k}:")                    
    # Print the arrival times in order of visit
    print("\nArrival times at each node in order of visit:")
    for t in range(len(d) - 1):
        for start in range(c):
            if start == 0:
                for end in range(c):
                    if sol.get_value(x[start, end, t, k]) == 1:
                        route = [(start, sol.get_value(T[start, k]))]
                        next_node = end
                        while next_node != 0:
                            route.append((next_node, sol.get_value(T[next_node, k])))
                            for next_end in range(c):
                                if sol.get_value(x[next_node, next_end, t, k]) == 1:
                                    next_node = next_end
                                    break
                        route.append(
                            (0, sol.get_value(T[c, k]))
                        )  # Use T[c] for the return to depot
                        if route[0][0] == 0 and route[-1][0] == 0:
                            for node, arrival in route:
                                print(f"Node {node}: {arrival:.2f}")
    
    print("\nCapacity for each node in order of visit:")
    for t in range(len(d) - 1):
        for start in range(c):
            if start == 0:
                for end in range(c):
                    if sol.get_value(x[start, end, t, k]) == 1:
                        route = [(start, sol.get_value(C[start, k]))]
                        next_node = end
                        while next_node != 0:
                            route.append((next_node, sol.get_value(C[next_node, k])))
                            for next_end in range(c):
                                if sol.get_value(x[next_node, next_end, t, k]) == 1:
                                    next_node = next_end
                                    break
                        route.append(
                            (0, sol.get_value(C[c - 1, k]))
                        )  # Use T[c] for the return to depot
                        if route[0][0] == 0 and route[-1][0] == 0:
                            for node, arrival in route:
                                print(f"Node {node}: {arrival:.2f}")

Total time required to collect/deliver all goods: 15.0 minutes
41.0

vehicle 0:

Route for time interval 0:

Route for time interval 1:
0 (arrivo: 540.00) -> 2 (arrivo: 545.00) -> 1 (arrivo: 550.00) -> 0 (arrivo: 555.00)

Route for time interval 2:

vehicle 1:

Route for time interval 0:

Route for time interval 1:
0 (arrivo: 540.00) -> 3 (arrivo: 545.00) -> 0 (arrivo: 555.00)

Route for time interval 2:

vehicle 0:

Arrival times at each node in order of visit:
Node 0: 540.00
Node 2: 545.00
Node 1: 550.00
Node 0: 555.00

Capacity for each node in order of visit:
Node 0: 0.00
Node 2: 10.00
Node 1: 20.00
Node 0: 0.00

vehicle 1:

Arrival times at each node in order of visit:
Node 0: 540.00
Node 3: 545.00
Node 0: 555.00

Capacity for each node in order of visit:
Node 0: 10.00
Node 3: 0.00
Node 0: 0.00
